<h3> Transformers

In [1]:
from transformers import pipeline

Tutaj są różne modele (i dane): https://huggingface.co/models 

<h4> Analiza wydźwięku (sentymentalna)

In [2]:
dokumenty = [
    "Słaby był ten film", 
    "such a great film", 
    ":(", 
    "I like this product but I won't buy it again", 
    "film był taki świetny, że już nigdy nie chcę go zobaczyć"
]

Model ocenia w skali od 1 do 5 pozytywność zdania :) 

- 1 (zdecydowanie negatywna), 

- 5 (zdecydowanie pozytywna).

In [3]:
sent_klasyfikator = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment") #zajmuje 648 M

In [4]:
sent_klasyfikator(dokumenty) #z 5 dokumentem jest problem :)

[{'label': '2 stars', 'score': 0.38626912236213684},
 {'label': '5 stars', 'score': 0.8855443000793457},
 {'label': '3 stars', 'score': 0.24814490973949432},
 {'label': '3 stars', 'score': 0.5084364414215088},
 {'label': '5 stars', 'score': 0.5227358937263489}]

Mozna tez dluższe recenzje, jednak do 512 tokenów

In [5]:
sent_klasyfikator("Też się zastanawiam skąd takie zachwyty. Horror z tego żaden. Oryginalności właściwie zero. Większość to klisze i schematy gatunku. Z wplecionym humorem. I jasne, reżyseria jest ok. Aktorsko również w porządku. Ale jako horror to słabo.")

[{'label': '1 star', 'score': 0.5370094180107117}]

In [6]:
sent_klasyfikator("Ciekawa fabuła,dobre efekty,klimat,świetna gra aktorska.Jestem bardzo pozytywnie zaskoczony ze film jest tak dobry.Oby kolejne czesci utrzymały tak wysoki poziom.")

[{'label': '4 stars', 'score': 0.5215557813644409}]

<h4> Generowanie podsumowań

In [7]:
tekst = ("""Bioinformatics is an interdisciplinary field that develops methods and software tools
for understanding biological data, in particular when the data sets are large and complex.
As an interdisciplinary field of science, bioinformatics combines biology, chemistry, physics,
computer science, information engineering, mathematics and statistics to analyze and interpret
the biological data. Bioinformatics has been used for in silico analyses of biological queries
using mathematical and statistical techniques. Bioinformatics includes biological studies that
use computer programming as part of their methodology, as well as specific analysis "pipelines"
that are repeatedly used, particularly in the field of genomics. Common uses of bioinformatics
include the identification of candidates genes and single nucleotide polymorphisms (SNPs). Often,
such identification is made with the aim to better understand the genetic basis of disease, unique
adaptations, desirable properties (esp. in agricultural species), or differences between populations.
In a less formal way, bioinformatics also tries to understand the organizational principles within
nucleic acid and protein sequences, called proteomics.
""")

In [8]:
podsumowanie = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6") #ponad 1G

In [9]:
podsumowanie(tekst, min_length=10, max_length=50)

[{'summary_text': ' Bioinformatics combines biology, chemistry, physics, computer science, mathematics and statistics to analyze and interpret biological data . Common uses include identification of candidates genes and single nucleotide polymorphisms (SNPs)'}]

<h4> Generowanie tekstu

In [10]:
tekst = "It was a dark night."

In [11]:
generator = pipeline("text-generation", model="gpt2") #zajmuje ponad 523 M

In [12]:
generator(tekst, max_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'It was a dark night. I felt like I had taken on a world of its own.\n\nIn that moment, even though we were all in denial about where we were heading — but then the truth dawned that it was indeed not really'}]

In [13]:
generator(tekst, max_length=100, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'It was a dark night. It was like Christmas. I didn\'t think I\'d see anybody anymore." "It\'s never right to leave that place. That\'s not how you want to live any longer. I\'m doing okay. But you want to live a longer life because you\'re out of a job." "No. I\'ve never done anything like this. I\'m lucky. And I\'m happy. I\'m happy to be leaving. If I did, I would have left. And'},
 {'generated_text': 'It was a dark night. Some were not quite so lucky. It must have been the brightest of them all, even a few hundred miles from their home.\n\nOne by one, those who were able to watch the clouds, came together to help out. The light of the sun rose, and when its golden rays were dispersed, a great crowd arose, and soon some, all gathered to watch. The sun, after giving her the opportunity of watching the clouds from the side of the road'},
 {'generated_text': "It was a dark night. The walls of the chamber were strewn with all kinds of smoke; I had heard it before, bu

<h4> Konwersacje

In [14]:
from transformers import Conversation

In [15]:
konwersator = pipeline("conversational", model="microsoft/DialoGPT-medium") #zajmuje ok 1G

In [18]:
konwersacja = Conversation("I would like to watch a nice horror. Do you recommend something?") #nie podaje tego jako zwykly tekst, tylko obiekt typu Conversation bo chce zeby byla pamietana cala historia

In [19]:
konwersator(konwersacja)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: ff11c22c-3b5d-4638-ad9f-80a20af7e46d 
user >> I would like to watch a nice horror. Do you recommend something? 
bot >> I recommend the movie The Ring. 

In [20]:
konwersacja.add_user_input("Great! Have you seen that film?")

In [21]:
konwersator(konwersacja)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: ff11c22c-3b5d-4638-ad9f-80a20af7e46d 
user >> I would like to watch a nice horror. Do you recommend something? 
bot >> I recommend the movie The Ring. 
user >> Great! Have you seen that film? 
bot >> I have not. 

<h4> Wypełnianie luki w tekście

In [33]:
luka = pipeline("fill-mask", model="bert-base-uncased") #ok 0.5G

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
luka(f"Hello, how {luka.tokenizer.mask_token} you")      

[{'sequence': 'hello, how are you',
  'score': 0.9426403045654297,
  'token': 2024,
  'token_str': 'are'},
 {'sequence': 'hello, how about you',
  'score': 0.021644026041030884,
  'token': 2055,
  'token_str': 'about'},
 {'sequence': 'hello, how could you',
  'score': 0.011218828149139881,
  'token': 2071,
  'token_str': 'could'},
 {'sequence': 'hello, how can you',
  'score': 0.0096061360090971,
  'token': 2064,
  'token_str': 'can'},
 {'sequence': 'hello, how may you',
  'score': 0.0027918575797230005,
  'token': 2089,
  'token_str': 'may'}]

In [24]:
 luka(f"Bioinformatics is {luka.tokenizer.mask_token} science")

[{'sequence': 'bioinformatics is a science',
  'score': 0.6050498485565186,
  'token': 1037,
  'token_str': 'a'},
 {'sequence': 'bioinformatics is computer science',
  'score': 0.22969503700733185,
  'token': 3274,
  'token_str': 'computer'},
 {'sequence': 'bioinformatics is information science',
  'score': 0.02828025259077549,
  'token': 2592,
  'token_str': 'information'},
 {'sequence': 'bioinformatics is life science',
  'score': 0.02481093257665634,
  'token': 2166,
  'token_str': 'life'},
 {'sequence': 'bioinformatics is the science',
  'score': 0.009441596455872059,
  'token': 1996,
  'token_str': 'the'}]

<h4> Odpowiadanie na pytania

In [25]:
odp = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad") #1.25G

Przykład 1

In [26]:
odp({
    "question": "What is Newton's third law of motion?",
    "context": "Newton's third law of motion states that, 'For every action there is equal and opposite reaction'"})

{'score': 0.3967142105102539,
 'start': 42,
 'end': 96,
 'answer': "'For every action there is equal and opposite reaction"}

Przykład 2

In [27]:
kontekst = (
    "The Lord of the Rings is an epic high fantasy novel by the English author and scholar J. R. R. Tolkien. Set in "
    "Middle-earth, the world at some distant time in the past, the story began as a sequel to Tolkien's 1937 children's book "
    "The Hobbit, but eventually developed into a much larger work. Written in stages between 1937 and 1949, The Lord of the "
    "Rings is one of the best-selling books ever written, with over 150 million copies sold. The title names the story's main "
    "antagonist, the Dark Lord Sauron, who had in an earlier age created the One Ring to rule the other Rings of Power as the "
    "ultimate weapon in his campaign to conquer and rule all of Middle-earth. From homely beginnings in the Shire, a hobbit "
    "land reminiscent of the English countryside, the story ranges across Middle-earth, following the quest to destroy the One "
    "Ring mainly through the eyes of the hobbits Frodo, Sam, Merry and Pippin. Although generally known to readers as a "
    "trilogy, the work was initially intended by Tolkien to be one volume of a two-volume set along with The Silmarillion, but "
    "this idea was dismissed by his publisher. For economic reasons, The Lord of the Rings was published in three volumes over "
    "the course of a year from 29 July 1954 to 20 October 1955. The three volumes were titled The Fellowship of the Ring, The "
    "Two Towers and The Return of the King. Structurally, the work is divided internally into six books, two per volume, with "
    "several appendices of background material at the end. Some editions print the entire work into a single volume, following "
    "the author's original intent. Tolkien's work, after an initially mixed reception by the literary establishment, has been "
    "the subject of extensive analysis of its themes and origins. Influences on this earlier work, and on the story of The "
    "Lord of the Rings, include philology, mythology, Christianity, earlier fantasy works, and his own experiences in the "
    "First World War. The Lord of the Rings in its turn has had a great effect on modern fantasy. The Lord of the Rings is "
    "considered by many one of the greatest fantasy books of all time.")

In [29]:
odp(question="Who is Frodo?", context=kontekst)

{'score': 0.6458943486213684, 'start': 869, 'end': 876, 'answer': 'hobbits'}

In [30]:
odp(question="What are the names of three volumes?", context=kontekst)

{'score': 0.846308708190918,
 'start': 1281,
 'end': 1350,
 'answer': 'The Fellowship of the Ring, The Two Towers and The Return of the King'}

<h4> Tłumaczenie pomiędzy językami

In [31]:
translator = pipeline("translation_pl_to_en", model="Helsinki-NLP/opus-mt-pl-en") #z pl na en, ok 300 M

In [32]:
translator("Wczoraj oglądałem fajny film.")

[{'translation_text': 'I was watching a cool movie last night.'}]

Ciekawostka: https://gadgets360.com/apps/news/google-translate-religious-texts-doomsday-prophecies-end-of-world-1888133

Oczywiście powyższe modele można mieszać między sobą :)